In [ ]:
import time
import statistics
import mysql.connector
import pandas as pd
import os
from dotenv import load_dotenv


In [ ]:
load_dotenv()

In [ ]:
mydb = mysql.connector.connect(
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    host=os.getenv('DB_HOST'),
    database=os.getenv('DB_NAME')
)
cursor = mydb.cursor()

In [ ]:
with open('Project 2\Queries\Index.sql', 'r') as f:
    ddl_script = f.read()

for statement in ddl_script.split(';'):
    stmt = statement.strip()
    if stmt:
        cursor.execute(stmt)

In [ ]:
print("Indexing successful!")

In [ ]:
queries_baseline_indexed={
    "Total Revenue by Customer": """
        SELECT
            c.customer_id,
            c.customer_city,
            c.customer_state,
            SUM(oi.price + oi.freight_value) AS total_revenue
        FROM
            customers c
            JOIN orders o ON c.customer_id = o.customer_id
            JOIN order_items oi ON o.order_id = oi.order_id
        WHERE
            oi.price IS NOT NULL
        GROUP BY
            c.customer_id,
            c.customer_city,
            c.customer_state
        ORDER BY
            total_revenue DESC;
    """,

    "Co-Purchased Product Pairs": """
        SELECT
            p1.product_name AS product_1,
            p2.product_name AS product_2,
            COUNT(*) AS times_bought_together
        FROM
            order_items o1
            JOIN order_items o2 ON o1.order_id = o2.order_id
                AND o1.product_id < o2.product_id
            JOIN products p1 ON o1.product_id = p1.product_id
            JOIN products p2 ON o2.product_id = p2.product_id
        WHERE
            p1.product_name != p2.product_name
            AND p1.product_name != '#N/A'
            AND p2.product_name != '#N/A'
        GROUP BY
            p1.product_name,
            p2.product_name
        ORDER BY
            times_bought_together DESC;
    """,

    "Customers by Days Since Last Order": """
        SELECT
            c.customer_id,
            c.customer_city,
            c.customer_state,
            MAX(o.order_approved_at) AS last_order_date,
            DATEDIFF(CURRENT_DATE, MAX(o.order_approved_at)) AS days_since_last_order
        FROM
            customers c
            JOIN orders o ON c.customer_id = o.customer_id
        WHERE
            o.order_approved_at IS NOT NULL
        GROUP BY
            c.customer_id,
            c.customer_city,
            c.customer_state
        ORDER BY
            days_since_last_order DESC;
    """,

    "Rank Sellers by Revenue and Sales within Product Categories and Regions": """
        SELECT
            s.seller_id,
            s.seller_state,
            s.seller_city,
            p.product_category_name,
            SUM(oi.price) AS total_revenue,
            COUNT(*) AS total_sales,
            RANK() OVER (
                PARTITION BY p.product_category_name, s.seller_state, s.seller_city
                ORDER BY SUM(oi.price) DESC
            ) AS seller_rank
        FROM
            order_items oi
            JOIN products p ON oi.product_id = p.product_id
            JOIN sellers s ON oi.seller_id = s.seller_id
        GROUP BY
            s.seller_id,
            s.seller_state,
            s.seller_city,
            p.product_category_name
        ORDER BY
            p.product_category_name,
            s.seller_state,
            s.seller_city,
            seller_rank;
    """,

    "Average Order Value by Payment Method and Installments": """
        SELECT
            payment_type,
            payment_installments,
            COUNT(*) AS total_transactions,
            SUM(payment_value) AS total_payment_value,
            ROUND(AVG(payment_value), 2) AS avg_order_value
        FROM
            order_payments
        GROUP BY
            payment_type,
            payment_installments
        ORDER BY
            payment_type,
            payment_installments;
    """,

    "Annual Breakdown of Orders, Revenue, Customers, Products, and Sellers": """
SELECT
    year(o.order_approved_at) AS year,
    count(oi.order_id) AS total_orders,
    (sum(price) + sum(freight_value)) AS total_revenue,
    count(DISTINCT o.customer_id) AS unique_customers,
    count(DISTINCT oi.product_id) AS unique_products,
    count(DISTINCT oi.seller_id) AS unique_sellers
FROM
    orders o
    INNER JOIN order_items oi ON o.order_id = oi.order_id
where
    order_approved_at is not null
group by
    year;
    """,
    "Top Customer and Seller Cities by Order Volume": """
        SELECT
    (
        SELECT
            s.seller_city
        FROM
            order_items oi
            JOIN sellers s ON oi.seller_id = s.seller_id
        GROUP BY
            s.seller_city
        HAVING
            COUNT(*) = (
                SELECT
                    MAX(city_count)
                FROM (
                    SELECT
                        s2.seller_city,
                        COUNT(*) AS city_count
                    FROM
                        order_items oi2
                        JOIN sellers s2 ON oi2.seller_id = s2.seller_id
                    GROUP BY
                        s2.seller_city
                ) AS seller_counts
            )
        LIMIT 1
    ) AS popular_seller_city,
    (
        SELECT
            c.customer_city
        FROM
            orders o
            JOIN order_items oi ON o.order_id = oi.order_id
            JOIN customers c ON o.customer_id = c.customer_id
        GROUP BY
            c.customer_city
        HAVING
            COUNT(*) = (
                SELECT
                    MAX(city_count)
                FROM (
                    SELECT
                        c2.customer_city,
                        COUNT(*) AS city_count
                    FROM
                        orders o2
                        JOIN order_items oi2 ON o2.order_id = oi2.order_id
                        JOIN customers c2 ON o2.customer_id = c2.customer_id
                    GROUP BY
                        c2.customer_city
                ) AS customer_counts
            )
        LIMIT 1
    ) AS popular_customer_city;
    """,

    "Compare Seller Revenue Against City Average in the Same City": """
        SELECT
            s.seller_id,
            s.seller_city,
            s.seller_state,
            SUM(oi.price) AS seller_revenue
        FROM
            sellers s
            JOIN order_items oi ON s.seller_id = oi.seller_id
        GROUP BY
            s.seller_id,
            s.seller_city,
            s.seller_state
        HAVING
            SUM(oi.price) > (
                SELECT AVG(city_totals.total_revenue)
                FROM (
                    SELECT
                        s2.seller_id,
                        SUM(oi2.price) AS total_revenue
                    FROM
                        sellers s2
                        JOIN order_items oi2 ON s2.seller_id = oi2.seller_id
                    WHERE
                        s2.seller_city = s.seller_city
                        AND s2.seller_id != s.seller_id
                    GROUP BY
                        s2.seller_id
                ) AS city_totals
            )
        ORDER BY
            seller_revenue DESC
        LIMIT 10;
    """,

    "Average, Max, and Min Delivery Time per Individual Product": """
        SELECT
            p.product_id,
            p.product_name,
            ROUND(AVG(DATEDIFF(o.order_delivered_customer_date, o.order_approved_at)), 2) AS avg_delivery_days,
            MAX(DATEDIFF(o.order_delivered_customer_date, o.order_approved_at)) AS max_delivery_days,
            MIN(DATEDIFF(o.order_delivered_customer_date, o.order_approved_at)) AS min_delivery_days,
            COUNT(*) AS total_orders
        FROM
            orders o
            JOIN order_items oi ON o.order_id = oi.order_id
            JOIN products p ON oi.product_id = p.product_id
        WHERE
            o.order_delivered_customer_date IS NOT NULL
            AND o.order_approved_at IS NOT NULL
            AND o.order_delivered_customer_date > o.order_approved_at
        GROUP BY
            p.product_id,
            p.product_name
        ORDER BY
            avg_delivery_days DESC;
    """,

    "Customer Segmentation Based on Order Count, Spending, and Recency": """
        SELECT
            o.customer_id,
            COUNT(DISTINCT o.order_id) AS order_count,
            SUM(oi.price + oi.freight_value) AS total_spent,
            DATEDIFF(CURRENT_DATE, MAX(o.order_approved_at)) AS days_since_last_order,
            CASE
                WHEN COUNT(DISTINCT o.order_id) = 1 THEN 'One-time Buyer'
                WHEN SUM(oi.price + oi.freight_value) >= 1000 THEN 'High-Value Buyer'
                WHEN COUNT(DISTINCT o.order_id) >= 5
                    AND DATEDIFF(CURRENT_DATE, MAX(o.order_approved_at)) <= 30 THEN 'Frequent & Recent Buyer'
                WHEN DATEDIFF(CURRENT_DATE, MAX(o.order_approved_at)) > 90 THEN 'Inactive Buyer'
                ELSE 'Regular Buyer'
            END AS customer_segment
        FROM
            orders o
            JOIN order_items oi ON o.order_id = oi.order_id
        WHERE
            o.order_approved_at IS NOT NULL
        GROUP BY
            o.customer_id
        ORDER BY
            total_spent DESC;
    """
}


In [ ]:
queries_optimized_indexed={
    "Optimized Total Revenue by Customer": """
        WITH order_revenue AS (
            SELECT
                order_id,
                SUM(price + freight_value) AS order_revenue
            FROM
                order_items
            WHERE
                price IS NOT NULL
            GROUP BY
                order_id
        )
        SELECT
            c.customer_id,
            c.customer_city,
            c.customer_state,
            SUM(orv.order_revenue) AS total_revenue
        FROM
            customers c
            INNER JOIN orders o ON o.customer_id = c.customer_id
            INNER JOIN order_revenue orv ON orv.order_id = o.order_id
        GROUP BY
            c.customer_id,
            c.customer_city,
            c.customer_state
        ORDER BY
            total_revenue DESC;
    """,

    "Optimized Co-Purchased Product Pairs": """
       WITH filtered_items AS (
  SELECT oi.order_id, oi.product_id, p.product_name
  FROM order_items oi
  JOIN products p ON oi.product_id = p.product_id
  WHERE p.product_name <> '#N/A'
),
pairs AS (
  SELECT
    f1.product_name   AS product_1,
    f2.product_name AS product_2
  FROM filtered_items f1
  JOIN filtered_items f2
    ON f1.order_id = f2.order_id
   AND f1.product_id < f2.product_id
)
SELECT
  product_1,
  product_2,
  COUNT(*) AS times_bought_together
FROM pairs
GROUP BY product_1, product_2
ORDER BY times_bought_together DESC;

    """,

    "Optimized Customers by Days Since Last Order": """
        WITH last_order AS (
            SELECT
                customer_id,
                MAX(order_approved_at) AS last_order_date
            FROM
                orders
            WHERE
                order_approved_at IS NOT NULL
            GROUP BY
                customer_id
        )
        SELECT
            c.customer_id,
            c.customer_city,
            c.customer_state,
            lo.last_order_date,
            DATEDIFF(CURRENT_DATE, lo.last_order_date) AS days_since_last_order
        FROM
            customers c
            JOIN last_order lo USING (customer_id)
        ORDER BY
            days_since_last_order DESC;
    """,

    "Optimized Rank Sellers by Revenue and Sales within Product Categories and Regions": """
        WITH seller_revenue AS (
            SELECT
                oi.seller_id,
                p.product_category_name,
                SUM(oi.price) AS total_revenue,
                COUNT(*) AS total_sales
            FROM
                order_items oi
                JOIN products p ON oi.product_id = p.product_id
            GROUP BY
                oi.seller_id,
                p.product_category_name
        )
        SELECT
            sr.seller_id,
            s.seller_state,
            s.seller_city,
            sr.product_category_name,
            sr.total_revenue,
            sr.total_sales,
            RANK() OVER (
                PARTITION BY sr.product_category_name, s.seller_state, s.seller_city
                ORDER BY sr.total_revenue DESC
            ) AS seller_rank
        FROM
            seller_revenue sr
            JOIN sellers s ON sr.seller_id = s.seller_id
        ORDER BY
            sr.product_category_name, s.seller_state, s.seller_city, seller_rank;
    """,

    "Optimized Average Order Value by Payment Method and Installments": """
        WITH filtered AS (
            SELECT
                payment_type,
                payment_installments,
                payment_value
            FROM
                order_payments
            WHERE
                payment_value IS NOT NULL
        )
        SELECT
            payment_type,
            payment_installments,
            COUNT(*) AS total_transactions,
            SUM(payment_value) AS total_payment_value,
            ROUND(SUM(payment_value) / COUNT(*), 2) AS avg_order_value
        FROM
            filtered
        GROUP BY
            payment_type,
            payment_installments
        ORDER BY
            payment_type,
            payment_installments;
    """,

    "Optimized Annual Breakdown of Orders, Revenue, Customers, Products, and Sellers": """
        WITH orders_valid AS (
            SELECT
                order_id,
                YEAR(order_approved_at) AS yr,
                customer_id
            FROM
                orders
            WHERE
                order_approved_at IS NOT NULL
        )
        SELECT
            ov.yr AS year,
            COUNT(oi.order_id) AS total_orders,
            SUM(oi.price + oi.freight_value) AS total_revenue,
            COUNT(DISTINCT ov.customer_id) AS unique_customers,
            COUNT(DISTINCT oi.product_id) AS unique_products,
            COUNT(DISTINCT oi.seller_id) AS unique_sellers
        FROM
            orders_valid ov
            JOIN order_items oi ON oi.order_id = ov.order_id
        GROUP BY
            ov.yr
        ORDER BY
            ov.yr;
    """,

    "Optimized Top Seller and Customer Cities by Order Volume": """
        WITH seller_ct AS (
            SELECT s.seller_city, COUNT(*) AS cnt
            FROM order_items oi
            JOIN sellers s ON oi.seller_id = s.seller_id
            GROUP BY s.seller_city
        ),
        cust_ct AS (
            SELECT c.customer_city, COUNT(*) AS cnt
            FROM orders o
            JOIN order_items oi ON o.order_id = oi.order_id
            JOIN customers c ON o.customer_id = c.customer_id
            GROUP BY c.customer_city
        )
        SELECT
            (SELECT seller_city FROM seller_ct ORDER BY cnt DESC LIMIT 1) AS popular_seller_city,
            (SELECT customer_city FROM cust_ct ORDER BY cnt DESC LIMIT 1) AS popular_customer_city;
    """,

    "Optimized Compare Seller Revenue Against Average Revenue in the Same City": """
        WITH city_avg AS (
            SELECT
                seller_city,
                AVG(total_revenue) AS avg_rev
            FROM (
                SELECT
                    s2.seller_city,
                    s2.seller_id,
                    SUM(oi2.price) AS total_revenue
                FROM
                    sellers s2
                    JOIN order_items oi2 ON s2.seller_id = oi2.seller_id
                GROUP BY
                    s2.seller_city, s2.seller_id
            ) t
            GROUP BY seller_city
        )
        SELECT
            s.seller_id,
            s.seller_city,
            s.seller_state,
            sr.seller_revenue
        FROM (
            SELECT
                seller_id,
                SUM(price) AS seller_revenue
            FROM
                order_items
            GROUP BY
                seller_id
        ) sr
        JOIN sellers s ON sr.seller_id = s.seller_id
        JOIN city_avg ca ON s.seller_city = ca.seller_city
        WHERE sr.seller_revenue > ca.avg_rev
        ORDER BY sr.seller_revenue DESC
        LIMIT 10;
    """,

    "Optimized Average, Max, and Min Delivery Time per Individual Product": """
        WITH order_date AS (
            SELECT
                oi.product_id,
                DATEDIFF(o.order_delivered_customer_date, o.order_approved_at) AS diff_days
            FROM
                orders o
                JOIN order_items oi ON o.order_id = oi.order_id
            WHERE
                o.order_delivered_customer_date > o.order_approved_at
        )
        SELECT
            p.product_id,
            p.product_name,
            ROUND(AVG(d.diff_days), 2) AS avg_delivery_days,
            MAX(d.diff_days) AS max_delivery_days,
            MIN(d.diff_days) AS min_delivery_days,
            COUNT(*) AS total_orders
        FROM
            order_date d
            JOIN products p ON d.product_id = p.product_id
        GROUP BY
            p.product_id,
            p.product_name
        ORDER BY
            avg_delivery_days DESC;
    """,

    "Optimized Customer Segmentation Based on Order Count, Spending, and Recency": """
        WITH customer_metrics AS (
            SELECT
                o.customer_id,
                COUNT(DISTINCT o.order_id) AS order_count,
                SUM(oi.price + oi.freight_value) AS total_spent,
                DATEDIFF(CURRENT_DATE, MAX(o.order_approved_at)) AS days_since_last_order
            FROM
                orders o
                JOIN order_items oi ON o.order_id = oi.order_id
            WHERE
                o.order_approved_at IS NOT NULL
            GROUP BY
                o.customer_id
        )
        SELECT
            customer_id,
            order_count,
            total_spent,
            days_since_last_order,
            CASE
                WHEN order_count = 1 THEN 'One-time Buyer'
                WHEN total_spent >= 1000 THEN 'High-Value Buyer'
                WHEN order_count >= 5 AND days_since_last_order <= 30 THEN 'Frequent & Recent Buyer'
                WHEN days_since_last_order > 90 THEN 'Inactive Buyer'
                ELSE 'Regular Buyer'
            END AS customer_segment
        FROM
            customer_metrics
        ORDER BY
            total_spent DESC;
    """
}

In [ ]:
indexed_baseline_results = []
final_baseline_indexed_results=[]
indexed_optimized_results = []
final_optimized_indexed_results=[]
N = 25

for name, sql in queries_baseline_indexed.items():
    runtimes = []
    for _ in range(N):
        start = time.perf_counter()
        cursor.execute(sql)
        cursor.fetchall()
        end = time.perf_counter()
        runtimes.append(end - start)

    indexed_baseline_results.append({
        "Query Name":   name,
        "Runs":         _+1,
        "Runtime of each iteration":   runtimes
    })
    final_baseline_indexed_results.append({
        "Query Name":   name,
        "Runs":         N,
        "Mean (sec)":   round(statistics.mean(runtimes), 4),
        "Standard Deviation (sec)": round(statistics.stdev(runtimes), 4)
    })

for name, sql in queries_optimized_indexed.items():
    runtimes = []
    for _ in range(N):
        start = time.perf_counter()
        cursor.execute(sql)
        cursor.fetchall()
        end = time.perf_counter()
        runtimes.append(end - start)

    indexed_optimized_results.append({
        "Query Name":   name,
        "Runs":         _+1,
        "Runtime of each iteration":   runtimes
    })
    final_optimized_indexed_results.append({
        "Query Name":   name,
        "Runs":         N,
        "Mean (sec)":   round(statistics.mean(runtimes), 4),
        "Standard Deviation (sec)": round(statistics.stdev(runtimes), 4)
    })
    
cursor.close()

In [ ]:
df = pd.DataFrame(final_optimized_indexed_results)
print(df.to_markdown(index=False))
df.to_csv('final_optimized_indexed_results.csv', index=False)